In [14]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)
options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)



load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

## Data partition

In [15]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [16]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    441     442 

## Models

In [17]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [18]:
model_rf <- train(x=TrainData, y=TrainClasses, 
               method = "ranger",
               trControl = ctrl) 

+ Fold01.Rep1: min.node.size=20, mtry=43, splitrule=gini 
- Fold01.Rep1: min.node.size=20, mtry=43, splitrule=gini 
+ Fold01.Rep1: min.node.size= 6, mtry=65, splitrule=extratrees 
- Fold01.Rep1: min.node.size= 6, mtry=65, splitrule=extratrees 
+ Fold01.Rep1: min.node.size= 7, mtry=45, splitrule=extratrees 
- Fold01.Rep1: min.node.size= 7, mtry=45, splitrule=extratrees 
+ Fold02.Rep1: min.node.size=20, mtry=43, splitrule=gini 
- Fold02.Rep1: min.node.size=20, mtry=43, splitrule=gini 
+ Fold02.Rep1: min.node.size= 6, mtry=65, splitrule=extratrees 
- Fold02.Rep1: min.node.size= 6, mtry=65, splitrule=extratrees 
+ Fold02.Rep1: min.node.size= 7, mtry=45, splitrule=extratrees 
- Fold02.Rep1: min.node.size= 7, mtry=45, splitrule=extratrees 
+ Fold03.Rep1: min.node.size=20, mtry=43, splitrule=gini 
- Fold03.Rep1: min.node.size=20, mtry=43, splitrule=gini 
+ Fold03.Rep1: min.node.size= 6, mtry=65, splitrule=extratrees 
- Fold03.Rep1: min.node.size= 6, mtry=65, splitrule=extratrees 
+ Fold03.Rep

In [19]:
model_rrf <- train(x=TrainData, y=TrainClasses, 
               method = "RRF",
               trControl = ctrl) 

+ Fold01.Rep1: mtry=31, coefReg=0.9085, coefImp=0.3314 
- Fold01.Rep1: mtry=31, coefReg=0.9085, coefImp=0.3314 
+ Fold01.Rep1: mtry=23, coefReg=0.5181, coefImp=0.5321 
- Fold01.Rep1: mtry=23, coefReg=0.5181, coefImp=0.5321 
+ Fold01.Rep1: mtry=22, coefReg=0.6770, coefImp=0.6618 
- Fold01.Rep1: mtry=22, coefReg=0.6770, coefImp=0.6618 
+ Fold02.Rep1: mtry=31, coefReg=0.9085, coefImp=0.3314 
- Fold02.Rep1: mtry=31, coefReg=0.9085, coefImp=0.3314 
+ Fold02.Rep1: mtry=23, coefReg=0.5181, coefImp=0.5321 
- Fold02.Rep1: mtry=23, coefReg=0.5181, coefImp=0.5321 
+ Fold02.Rep1: mtry=22, coefReg=0.6770, coefImp=0.6618 
- Fold02.Rep1: mtry=22, coefReg=0.6770, coefImp=0.6618 
+ Fold03.Rep1: mtry=31, coefReg=0.9085, coefImp=0.3314 
- Fold03.Rep1: mtry=31, coefReg=0.9085, coefImp=0.3314 
+ Fold03.Rep1: mtry=23, coefReg=0.5181, coefImp=0.5321 
- Fold03.Rep1: mtry=23, coefReg=0.5181, coefImp=0.5321 
+ Fold03.Rep1: mtry=22, coefReg=0.6770, coefImp=0.6618 
- Fold03.Rep1: mtry=22, coefReg=0.6770, coefImp=

## Results

In [20]:
print(model_rf)
plot(model_rf)

Random Forest 

883 samples
 73 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 795, 794, 795, 794, 795, 795, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   Accuracy   Kappa    
   6             65    extratrees  0.7849208  0.5698419
   7             45    extratrees  0.7916879  0.5833966
  20             43    gini        0.7746808  0.5493536

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 45, splitrule = extratrees
 and min.node.size = 7.


In [21]:
print(model_rrf)
plot(model_rrf) 

Regularized Random Forest 

883 samples
 73 predictor
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 795, 794, 795, 795, 795, 795, ... 
Resampling results across tuning parameters:

  mtry  coefReg    coefImp    Accuracy   Kappa    
  22    0.6769897  0.6617693  0.7100613  0.4200870
  23    0.5181322  0.5320754  0.7372574  0.4744856
  31    0.9084504  0.3313986  0.7700715  0.5400200

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were mtry = 31, coefReg = 0.9084504
 and coefImp = 0.3313986.


In [22]:
# collect resamples
results <- resamples(list(ranger=model_rf,rrf=model_rrf))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: ranger, rrf 
Number of resamples: 10 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ranger 0.7415730 0.7642045 0.7865169 0.7916879 0.8153409 0.8522727    0
rrf    0.7045455 0.7436798 0.7897727 0.7700715 0.7977528 0.8068182    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
ranger 0.4832113 0.5284091 0.5731044 0.5833966 0.6306818 0.7045455    0
rrf    0.4090909 0.4868205 0.5795455 0.5400200 0.5953523 0.6136364    0


## Prediction

In [23]:
pred_rf <- predict(model_rf,TestData)  
pred_rf.prob <- predict(model_rf,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_rf) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      67      60
   NotFast   39     213
                                          
               Accuracy : 0.7388          
                 95% CI : (0.6915, 0.7823)
    No Information Rate : 0.7203          
    P-Value [Acc > NIR] : 0.22963         
                                          
                  Kappa : 0.3887          
                                          
 Mcnemar's Test P-Value : 0.04442         
                                          
            Sensitivity : 0.6321          
            Specificity : 0.7802          
         Pos Pred Value : 0.5276          
         Neg Pred Value : 0.8452          
             Prevalence : 0.2797          
         Detection Rate : 0.1768          
   Detection Prevalence : 0.3351          
      Balanced Accuracy : 0.7061          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.6320755            0.7802198            0.5275591 
      Neg Pred Value            Precision               Recall 
           0.8452381            0.5275591            0.6320755 
                  F1           Prevalence       Detection Rate 
           0.5751073            0.2796834            0.1767810 
Detection Prevalence    Balanced Accuracy 
           0.3350923            0.7061476

In [24]:
roc_obj <- roc(TestClasses, as.numeric(pred_rf))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [25]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_rf))

Data: as.numeric(pred_rf) in 127 controls (TestClasses fast) < 252 cases (TestClasses NotFast).
Area under the curve: 0.6864

In [26]:
pred_rf.prob

fast,NotFast
0.7897476,0.21025238
0.2441000,0.75590000
0.2248952,0.77510476
0.5596667,0.44033333
0.3304952,0.66950476
0.4177048,0.58229524
0.3329524,0.66704762
0.3589905,0.64100952
0.4465762,0.55342381
0.3291333,0.67086667
